# Project - Hospital Database

**Copyright: © NexStream Technical Education, LLC**.  
All rights reserved

---
In this assignment you will wrap up this unit by designing a database for a fictitious hospital and track several interactions with patients, staff, and billing cycles.   Please refer to the lecture units and the reference Colab script below for instructions and details on each of the database commands needed to complete this assignment.

Follow the steps below and record your output - you will need to upload it as part of your assignment.
Please refer to the previous lecture units on simple, multi-table, sub-queries, and summary queries for more details on each step.


In [16]:
#Step 1
#   Mount your drive and change to the folder.
#   Note if get ERROR:root:Internal Python error in the inspect module error when trying to mount drive, restart runtime

# Mount drive

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# Modify the following change directory (cd) command to point to your working folder
# %cd /content/drive/My\ Drive/Colab\ Notebooks/MLF-300/ML-330-Databases

ModuleNotFoundError: No module named 'google'

In [35]:
#Note if get ERROR:root:Internal Python error in the inspect module error when trying to mount drive, restart runtime
#If restarted runtime, then execute previous cell!!

#Step 1:  Project setup and generic query execution function
#     1a: Import the sqlite3 library
import sqlite3
import os

#     1b: Connect to a database, call it 'myHospital.db'
#         Hint:  https://docs.python.org/3/library/sqlite3.html#connection-objects
#     YOUR CODE HERE...

data_path = '/Users/magnus/Desktop/code-projects/187818-PythonFundamentalsandApplications/Module_4.4_Assignment2/'
filename = 'myHospital'

db = sqlite3.connect(data_path + filename + '.sqlite3')

#     1c: Use the following generic query execution function
#         API:   def exe_query(query):
#         Input: query strin
#         Return: cursor contents
#         Hint:  https://docs.python.org/3/library/sqlite3.html#cursor-objects
#                Setup cursor, execute query, fetch the contents
def exe_query(query):
  cursor = db.execute(query)
  return cursor.fetchall()




In [36]:
#Step 2:
#   Create tables for Bills, Patients, Rooms, Staffs, Surgeries, SurgeryStaffs
#   Hint - use CREATE TABLE IF NOT EXISTS in case the table has already been created.

#   2a:  Bills
db.execute('''
CREATE TABLE IF NOT EXISTS Bills (
	ssn TEXT NOT NULL DEFAULT '000-00-0000', --comment: ssn will be 000-00-0000 if not known
	paid INTEGER,
	total INTEGER
);''')
db.commit()


#   2b:  Patients - create the Patients table with the following fields
#	        ssn TEXT NOT NULL,
#        	firstName VARCHAR(15) NOT NULL,
#        	lastName VARCHAR(15) NOT NULL,
#        	gender TEXT NOT NULL,
#        	dob DATE NOT NULL,
#        	phone VARCHAR(20) NOT NULL,
#        	address VARCHAR(100),
#        	primary key (ssn)
# YOUR CODE HERE...
db.execute(''' 
CREATE TABLE IF NOT EXISTS Patients (
  ssn VARCHAR(100) NOT NULL PRIMARY KEY,
  firstName VARCHAR(15) NOT NULL,
  lastName VARCHAR(15) NOT NULL,
  gender VARCHAR(100) NOT NULL,
  dob DATE NOT NULL,
  phone VARCHAR(20) NOT NULL,
  address VARCHAR(100)
);''')
db.commit()
#   2c:  Rooms - create the Rooms table with the following fields
#                Note, the '--' is a comment in the query
#	        id,
#        	type INTEGER,       -- 1 emergency room, 2 surgeon room, 3 wards
#        	location INTEGER,
#        	ssn TEXT REFERENCES Patients (ssn) ON DELETE SET NULL,  -- if empty, then it is available
#        	PRIMARY KEY (id)

# YOUR CODE HERE...

db.execute(''' 
CREATE TABLE IF NOT EXISTS Rooms (
  id INTEGER NOT NULL PRIMARY KEY,
  type INTEGER,       -- 1 emergency room, 2 surgeon room, 3 wards (salas)
  location INTEGER,
  ssn VARCHAR(100) REFERENCES Patients (ssn) ON DELETE SET NULL  -- if empty, then it is available
);''')
db.commit()
#   2d:  Staffs - create the Staffs table with the following fields
#                 Note, the '--' is a comment in the query
#	        id INTEGER NOT NULL,
#	        firstName VARCHAR(15) NOT NULL,
#	        lastName VARCHAR(15) NOT NULL,
#	        type INTEGER,       -- 1 administoartor staffs; 2 doctors; 3 nurses; 4t surgeons
#        	phone VARCHAR(20) NOT NULL,
#        	department VARCHAR(100),
#	        PRIMARY KEY (id)

# YOUR CODE HERE...

db.execute(''' 
CREATE TABLE IF NOT EXISTS Staffs (
  id INTEGER NOT NULL,
  firstName VARCHAR(15) NOT NULL,
  lastName VARCHAR(15) NOT NULL,
  type INTEGER,       -- 1 administoartor staffs; 2 doctors; 3 nurses; 4t surgeons
  phone VARCHAR(20) NOT NULL,
  department VARCHAR(100),
  PRIMARY KEY (id)
);''')
db.commit()
#   2e:  Surgeries - create the Surgeries table with the following fields
#                    Note, the '--' is a comment in the query
#	        id INTEGER NOT NULL,
#        	ssn TEXT NOT NULL,          -- patient's ssn
#        	emergency INTEGER NOT NULL, -- emergency 1, regular surgey 2
#        	room INTEGER NOT NULL,      -- surgery room id
#	        checkIn DATETIME NOT NULL,
#	        checkOut DATETIME,          -- if null, it is occpuied
#	        PRIMARY KEY (id)

# YOUR CODE HERE...

# SURGERIES TABLE 
db.execute(''' 
CREATE TABLE IF NOT EXISTS Surgeries( 
	id INTEGER NOT NULL,
	ssn VARCHAR(100) NOT NULL,          -- patient's ssn
	emergency INTEGER NOT NULL, -- emergency 1, regular surgey 2
	room INTEGER NOT NULL,      -- surgery room id
	checkIn DATETIME NOT NULL,
	checkOut DATETIME,          -- if null, it is occpuied
	PRIMARY KEY (id)
);''')
db.commit()
#   2f:  SurgeryStaffs - create the SurgeryStaffs table with the following fields
#       	surgeryId INTEGER NOT NULL,
#       	surgeonId INTEGER NOT NULL,
#       	PRIMARY KEY (surgeryId, surgeonId)

# YOUR CODE HERE...

# SURGERY STAFF TABLE 
db.execute(''' 
CREATE TABLE IF NOT EXISTS SurgeryStaffs( 
	surgeryId INTEGER NOT NULL,
	surgeonId INTEGER NOT NULL,
    PRIMARY KEY (surgeryId, surgeonId) 
);''')
db.commit()

#   2f   List tables
cursor = db.execute('''SELECT name FROM sqlite_master WHERE type='table';''' )
print(cursor.fetchall())

#Insert records into each of the tables
#First, delete all entries to avoid adding duplicate records if rerunning this cell
db.execute('''DELETE FROM Bills;''')
db.execute('''DELETE FROM Patients;''')
db.execute('''DELETE FROM Rooms;''')
db.execute('''DELETE FROM Staffs;''')
db.execute('''DELETE FROM Surgeries;''')
db.execute('''DELETE FROM SurgeryStaffs;''')


[('Bills',), ('Patients',), ('Rooms',), ('Staffs',), ('Surgeries',), ('SurgeryStaffs',)]


In [37]:
#Step 3:  Check in patients from provided shared files

#   3a:  Read Patients Table from file
#YOUR CODE HERE...
f = open("insertPatients.sql")
line = f.readline()
while line:
  db.execute(line)
  line = f.readline()
f.close()
db.commit()

# Step 3b
# Manually check-in a patient with the following profile:
#   ssn: '100-22-1235',
# 	first name: 'Mike',
# 	last name 'Lopez',
#   gender 'Male',
# 	dob: '1985-05-19',
# 	telphone: 8581424098,
# 	address: '590 Mapple St, San Diego, 92122'
# Save your query statement to variables named per the instructions below.
# Your query will be executed and contents will be checked in a doctest module.

#query_insert_patients =   YOUR CODE HERE...

query_insert_patients = ''' 
INSERT INTO Patients VALUES('100-22-1235','Mike','Lopez','Male','1985-05-19',8581424098,'590 Mapple St, San Diego, 92122')
'''
exe_query(query_insert_patients)
db.commit()

# Step 3c:  printout the Patients table
# Save your query statement to variables named per the instructions below. Your query will be executed and contents will be checked in a doctest module.
#query_select_patients =   YOUR CODE HERE...
query_select_patients = 'SELECT * FROM Patients;'
patients = '\n'.join([str(row) for row in exe_query(query_select_patients)])
print('---- PATIENTS ----\n', patients)


---- PATIENTS ----
 ('100-22-1234', 'Olivia', 'Smith', 'Male', '2005-05-19', '8581234567', '120 Main St, San Diego, 92121')
('100-22-1236', 'James', 'Thomas', 'Male', '1965-07-01', '61930993089', '6207 Rose St, San Diego, 92121')
('100-22-1237', 'Elijah', 'Harris', 'Female', '1945-07-01', '2618300749', '2007 Plain St, San Diego, 92131')
('100-22-1238', 'Harper', 'Garcia', 'Female', '1930-07-01', '5263800274', '1257 Washington St, San Diego, 92130')
('100-22-1235', 'Mike', 'Lopez', 'Male', '1985-05-19', '8581424098', '590 Mapple St, San Diego, 92122')


In [38]:
# Step 4 insert data into other tables:  Bills, Rooms, Staffs, Surgeries, SurgeryStaffs
# Step 4a:  Bills
f = open("insertBills.sql")
line = f.readline()
while line:
  db.execute(line)
  line = f.readline()
f.close()
db.commit()

# Step 4b:  Rooms
#   YOUR CODE HERE...
f = open("insertRooms.sql")
line = f.readline()
while line:
  db.execute(line)
  line = f.readline()
f.close()
db.commit()

# Step 4c:  Staffs
f = open("insertStaffs.sql")
line = f.readline()
while line:
  db.execute(line)
  line = f.readline()
f.close()
db.commit()

# Step 4d:  Surgeries
f = open("insertSurgeries.sql")
line = f.readline()
while line:
  db.execute(line)
  line = f.readline()
f.close()
db.commit()

# Step 4e:  SurgeryStaffs
f = open("insertSurgeryStaffs.sql")
line = f.readline()
while line:
  db.execute(line)
  line = f.readline()
f.close()
db.commit()


# Step 4f:  Printout all the tables after inserting the records
# Save your query statement to variables named per the instructions below. Your query will be executed and contents will be checked in a doctest module.

query_select_bills = 'SELECT * FROM Bills;'
bills = '\n'.join([str(row) for row in exe_query(query_select_bills)])
print('---- Bills ----\n', bills)

query_select_rooms = 'SELECT * FROM Rooms;'
rooms = '\n'.join([str(row) for row in exe_query(query_select_rooms)])
print('---- Rooms ----\n', rooms)

query_select_staffs = 'SELECT * FROM Staffs;'
staffs = '\n'.join([str(row) for row in exe_query(query_select_staffs)])
print('---- Staffs ----\n', staffs)

query_select_surgeries =  'SELECT * FROM Surgeries;'
surgeries = '\n'.join([str(row) for row in exe_query(query_select_surgeries)])
print('---- Surgeries ----\n', surgeries)

query_select_surgeryStaffs = 'SELECT * FROM SurgeryStaffs;'
surgeryStaffs = '\n'.join([str(row) for row in exe_query(query_select_surgeryStaffs)])
print('---- SurgeryStaffs ----\n', surgeryStaffs)


---- Bills ----
 ('100-22-1234', 50, 100)
('100-22-1235', 0, 1000)
('100-22-1236', 100, 3000)
---- Rooms ----
 (1, 1, 'emergency room area 1', None)
(2, 2, 'surgery room area 1', '100-22-1234')
(3, 2, 'surgery room area 2', '100-22-1235')
(4, 3, 'ward room area 1', '100-22-1235')
(5, 3, 'ward room area 2', None)
(6, 3, 'ward room area 3', '100-22-1234')
---- Staffs ----
 (100000, 'Emma', 'Johnson', 1, '858-293-3097', 'Administration dept')
(100001, 'Emma', 'Johnson', 1, '858-903-3148', 'Administration dept')
(100100, 'Ava', 'Miller', 2, '858-203-3800', 'Heart dept')
(100101, 'Ethan', 'Johnson', 2, '248-726-3948', 'Heart dept')
(100121, 'Amelia', 'Johnson', 2, '858-726-3948', 'ENT dept')
(100202, 'Sophia', 'Davis', 3, '858-203-1848', 'Nurse dept')
(100203, 'Isabella', 'Brown', 4, '858-256-3970', 'Surgeon dept 3')
(100222, 'Mia', 'Williams', 3, '308-203-1848', 'Nurse dept')
(100300, 'James', 'Taylor', 4, '619-203-1949', 'Surgeon dept 1')
(100301, 'Lucas', 'Millor', 4, '219-207-9830', 'Su

In [39]:
# Step 5 Check out patients.  Do two operations:
#   1.  Remove from Patients table
#   2.  Release the room resources
#   Save your query statement to variables named per the instructions below.
#  Your query will be executed and contents will be checked in a doctest module.

#   5a:
#   Check out patient with ssn: '100-22-1235'
query_checkout_patient= "DELETE FROM Patients WHERE ssn= '100-22-1235';"
exe_query(query_checkout_patient)
db.commit()

#   5b
#   Release the room resource
query_release_room = "UPDATE Rooms SET ssn = NULL WHERE ssn = '100-22-1235';" 
exe_query(query_release_room)
db.commit()

#   5c:  printout the Patients and Rooms tables after checkout
query_select_patients = 'SELECT * FROM PATIENTS;'
patients_after_checkout = '\n'.join([str(row) for row in exe_query(query_select_patients)])
print('\n---- PATIENTS (after patient checkout) ----\n', patients_after_checkout)

query_select_rooms = 'SELECT * FROM Rooms;'
rooms_after_checkout = '\n'.join([str(row) for row in exe_query(query_select_rooms)])
print('\n---- ROOMS (after patient checkout) ----\n', rooms_after_checkout)



---- PATIENTS (after patient checkout) ----
 ('100-22-1234', 'Olivia', 'Smith', 'Male', '2005-05-19', '8581234567', '120 Main St, San Diego, 92121')
('100-22-1236', 'James', 'Thomas', 'Male', '1965-07-01', '61930993089', '6207 Rose St, San Diego, 92121')
('100-22-1237', 'Elijah', 'Harris', 'Female', '1945-07-01', '2618300749', '2007 Plain St, San Diego, 92131')
('100-22-1238', 'Harper', 'Garcia', 'Female', '1930-07-01', '5263800274', '1257 Washington St, San Diego, 92130')

---- ROOMS (after patient checkout) ----
 (1, 1, 'emergency room area 1', None)
(2, 2, 'surgery room area 1', '100-22-1234')
(3, 2, 'surgery room area 2', None)
(4, 3, 'ward room area 1', None)
(5, 3, 'ward room area 2', None)
(6, 3, 'ward room area 3', '100-22-1234')


In [40]:
#Step 6: Queries

#Step 6a:  How many patients are in the hospital?
query_select_patients_count = "SELECT COUNT(*) FROM Patients;"
patients_count = '\n'.join([str(row) for row in exe_query(query_select_patients_count)])
print('\n---- PATIENTS (count) ----\n', patients_count)

#Step 6b:  How many staff are in the hospital?
query_select_staff_count = "SELECT COUNT(*) FROM Staffs;"
staff_count = '\n'.join([str(row) for row in exe_query(query_select_staff_count)])
print('\n---- STAFFS (total count) ----\n', staff_count)

#Step 6c:  How many doctors are in the hospital?
query_select_doctors_count =  "SELECT COUNT(*) FROM Staffs WHERE type = 2;"
doctors_count = '\n'.join([str(row) for row in exe_query(query_select_doctors_count)])
print('\n---- STAFFS (total doctors count) ----\n', doctors_count)

#Step 6d:  How many beds are available in the wards?
query_select_beds_available = "SELECT COUNT(*) FROM Rooms WHERE type = 3 AND ssn IS NULL;"
beds_count = '\n'.join([str(row) for row in exe_query(query_select_beds_available)])
print('\n---- ROOMS (total ward beds available) ----\n', beds_count);


---- PATIENTS (count) ----
 (4,)

---- STAFFS (total count) ----
 (12,)

---- STAFFS (total doctors count) ----
 (3,)

---- ROOMS (total ward beds available) ----
 (2,)


In [41]:

#Step 7:  Which surgeon(s) did surgery on patient 100-22-1235
#   Hint:  Use single query using INNER JOIN
query_select_surgeons = """
SELECT DISTINCT s.id, s.firstname, s.lastName, s.type, s.phone, s.department
FROM Staffs s
INNER JOIN SurgeryStaffs ss ON s.id = ss.surgeonId
INNER JOIN Surgeries su ON ss.surgeryId = su.id
WHERE su.ssn = '100-22-1235'
ORDER BY s.id;
"""
surgeons_operating_on_patient = '\n'.join([str(row) for row in exe_query(query_select_surgeons)])
print('\n---- Surgeons who operated on 100-22-1235 (Query Option 2) ----\n', surgeons_operating_on_patient);

#-------------------------------------------------------------------------------------------------
#Test with the following doctest test vectors.
#DO NOT EDIT THE TEST CODE!!!!
#Even changing the spacing can cause errors.
#The test code will automatically execute when you run the cell.
#You should test all your combination of outputs but your code at least must pass these exact tests.
#If your code fails, you will see a description in the console cell.
import doctest

print('\n\n************* Doctest output ********************')
"""

  >>> print('step3b_patients:', patients)
  step3b_patients: ('100-22-1234', 'Olivia', 'Smith', 'Male', '2005-05-19', '8581234567', '120 Main St, San Diego, 92121')
  ('100-22-1236', 'James', 'Thomas', 'Male', '1965-07-01', '61930993089', '6207 Rose St, San Diego, 92121')
  ('100-22-1237', 'Elijah', 'Harris', 'Female', '1945-07-01', '2618300749', '2007 Plain St, San Diego, 92131')
  ('100-22-1238', 'Harper', 'Garcia', 'Female', '1930-07-01', '5263800274', '1257 Washington St, San Diego, 92130')
  ('100-22-1235', 'Mike', 'Lopez', 'Male', '1985-05-19', '8581424098', '590 Mapple St, San Diego, 92122')
  >>> print('step4f_bills:', bills)
  step4f_bills: ('100-22-1234', 50, 100)
  ('100-22-1235', 0, 1000)
  ('100-22-1236', 100, 3000)
  >>> print('step4f_rooms:', rooms)
  step4f_rooms: (1, 1, 'emergency room area 1', None)
  (2, 2, 'surgery room area 1', '100-22-1234')
  (3, 2, 'surgery room area 2', '100-22-1235')
  (4, 3, 'ward room area 1', '100-22-1235')
  (5, 3, 'ward room area 2', None)
  (6, 3, 'ward room area 3', '100-22-1234')
  >>> print('step4f_staffs:', staffs)
  step4f_staffs: (100000, 'Emma', 'Johnson', 1, '858-293-3097', 'Administration dept')
  (100001, 'Emma', 'Johnson', 1, '858-903-3148', 'Administration dept')
  (100100, 'Ava', 'Miller', 2, '858-203-3800', 'Heart dept')
  (100101, 'Ethan', 'Johnson', 2, '248-726-3948', 'Heart dept')
  (100121, 'Amelia', 'Johnson', 2, '858-726-3948', 'ENT dept')
  (100202, 'Sophia', 'Davis', 3, '858-203-1848', 'Nurse dept')
  (100203, 'Isabella', 'Brown', 4, '858-256-3970', 'Surgeon dept 3')
  (100222, 'Mia', 'Williams', 3, '308-203-1848', 'Nurse dept')
  (100300, 'James', 'Taylor', 4, '619-203-1949', 'Surgeon dept 1')
  (100301, 'Lucas', 'Millor', 4, '219-207-9830', 'Surgeon dept 1')
  (100302, 'Ethan', 'Evans', 4, '619-702-0395', 'Surgeon dept 2')
  (100303, 'Charlotte', 'Brown', 4, '626-243-1960', 'Surgeon dept 2')
  >>> print('step4f_surgeries:', surgeries)
  step4f_surgeries: (1, '100-22-1234', 1, 10, '2019-01-05', '2019-01-06')
  (2, '100-22-1235', 2, 100, '2020-09-11', '2020-09-12')
  (3, '100-22-1235', 2, 100, '2020-10-01', '2020-10-02')
  (4, '100-22-1236', 2, 100, '2020-10-01', '2020-10-02')
  >>> print('step4f_surgeryStaffs:', surgeryStaffs)
  step4f_surgeryStaffs: (1, 100300)
  (2, 100301)
  (2, 100300)
  (3, 100300)
  (3, 100301)
  (4, 100301)
  (4, 100303)
  (4, 100300)
  >>> print('step5c_patients_after_checkout:', patients_after_checkout)
  step5c_patients_after_checkout: ('100-22-1234', 'Olivia', 'Smith', 'Male', '2005-05-19', '8581234567', '120 Main St, San Diego, 92121')
  ('100-22-1236', 'James', 'Thomas', 'Male', '1965-07-01', '61930993089', '6207 Rose St, San Diego, 92121')
  ('100-22-1237', 'Elijah', 'Harris', 'Female', '1945-07-01', '2618300749', '2007 Plain St, San Diego, 92131')
  ('100-22-1238', 'Harper', 'Garcia', 'Female', '1930-07-01', '5263800274', '1257 Washington St, San Diego, 92130')
  >>> print('step5c_rooms_after_checkout:', rooms_after_checkout)
  step5c_rooms_after_checkout: (1, 1, 'emergency room area 1', None)
  (2, 2, 'surgery room area 1', '100-22-1234')
  (3, 2, 'surgery room area 2', None)
  (4, 3, 'ward room area 1', None)
  (5, 3, 'ward room area 2', None)
  (6, 3, 'ward room area 3', '100-22-1234')
  >>> print('step6a_patients_count:', patients_count)
  step6a_patients_count: (4,)
  >>> print('step6b_staff_count:', staff_count)
  step6b_staff_count: (12,)
  >>> print('step6c_doctors_count:', doctors_count)
  step6c_doctors_count: (3,)
  >>> print('step6d_beds_count:', beds_count)
  step6d_beds_count: (2,)
  >>> print('step7_operating_surgeons:', surgeons_operating_on_patient)
  step7_operating_surgeons: (100300, 'James', 'Taylor', 4, '619-203-1949', 'Surgeon dept 1')
  (100301, 'Lucas', 'Millor', 4, '219-207-9830', 'Surgeon dept 1')
"""


doctest.testmod(verbose=True)
db.close()


---- Surgeons who operated on 100-22-1235 (Query Option 2) ----
 (100300, 'James', 'Taylor', 4, '619-203-1949', 'Surgeon dept 1')
(100301, 'Lucas', 'Millor', 4, '219-207-9830', 'Surgeon dept 1')


************* Doctest output ********************
Trying:
    print('step3b_patients:', patients)
Expecting:
    step3b_patients: ('100-22-1234', 'Olivia', 'Smith', 'Male', '2005-05-19', '8581234567', '120 Main St, San Diego, 92121')
    ('100-22-1236', 'James', 'Thomas', 'Male', '1965-07-01', '61930993089', '6207 Rose St, San Diego, 92121')
    ('100-22-1237', 'Elijah', 'Harris', 'Female', '1945-07-01', '2618300749', '2007 Plain St, San Diego, 92131')
    ('100-22-1238', 'Harper', 'Garcia', 'Female', '1930-07-01', '5263800274', '1257 Washington St, San Diego, 92130')
    ('100-22-1235', 'Mike', 'Lopez', 'Male', '1985-05-19', '8581424098', '590 Mapple St, San Diego, 92122')
ok
Trying:
    print('step4f_bills:', bills)
Expecting:
    step4f_bills: ('100-22-1234', 50, 100)
    ('100-22-1235', 